# German stopwords (Bunderregierung)

In [1]:
# Data processing
import json
import numpy as np
import pandas as pd
import re

# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import stopwords

#getting german stopwords:
german_stop_words = stopwords.words('german')

# Funcoes para limpar as palavras:

def remove_stopwords(element):
    if isinstance(element, str):
        if element in german_stop_words:
            return ''
        else:
            return element

def replace_char(x):
  return re.sub('[^a-zA-Z\süöäß]', '', x)

def clean_corpus(s):
    s = s.apply(lambda x: x.lower() if isinstance(x, str) else x)     #tolower:
    idx = s.apply(lambda x: isinstance(x, str))                       # Criar um índice booleano para filtrar os valores não-caracteres
    s = s.drop(idx[~idx].index)                                       # Remover os valores não-caracteres
    s = s.apply(replace_char)                                         # remover as partes de palavras que nao sao caracteres ou espacos
    idx = s.apply(lambda x: len(x) < 2)                               # Criar um índice booleano para filtrar os elementos com comprimento menor que 2
    s = s.drop(idx[idx].index)                                        # Remover os elementos com comprimento menor que 2
    s = s.apply(remove_stopwords)                                     # remover as stopwords: 
    idx = s.apply(lambda x: x == '')                                  # Criar um índice booleano para filtrar os elementos vazios
    s = s.drop(idx[idx].index)                                        # Remover os elementos vazios
    
    return s

def frequency_words(x):
    # Cria um dicionário vazio para armazenar os contadores
    contadores = {} 
    
    for palavra in x:
        if palavra in contadores:                   # Se a palavra já estiver no dicionário, incrementa o contador
            contadores[palavra] += 1
        else:                                       # Se a palavra não estiver no dicionário, adiciona-a e define o contador como 1
            contadores[palavra] = 1

    return contadores

# Abrindo o arquivo:

df_original = pd.read_json('C:\\Users\\opc\\Documents\\Python_Scripts\\new_champions\\01_gesetze_sammlung\\de_concatenado.json')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\opc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\opc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\opc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Considerations

### Stopwords

I'm going to separate the documents per thema, as I did for the european Stopwords.
I will add to stopwords:

1. Stopwords that refers to Austria
2. Pronouns that might scape the original stopword list

Since we are not working with laws, I don'T expect that terms related to law (i.e. "Absatzt") will play a role here

### Document division

I'm going to divide the Austrian and German normative documents as:

- Daten and KI documents
- Timber and Circular economy / Green Deal documents
- Regierungsprogramm

The reason for that is that the last one has no specific thema, but should mention all the government's subthemas.

In [9]:
#df_original['documento'].unique()

# klima and timber
df_kl_timber_de = df_original.query("documento == 'gesetz_klimaziele' | documento == 'leitlinien_kreislaufwirtschaft'  | documento == 'roadmap_2025_holzwirtschaft' ")


#Daten und KI
df_daten_ki_de = df_original.query("documento == 'nationale_datenstrategie'  | documento ==  'nationale_ki_strategie_addendum'  | documento ==  'nationale_ki_strategie'")

#Regierungsprogramm
df_reg_de = df_original.query("documento =='koalitionsvertrag_regierungsprogramm'")

In [6]:
### 1. Olhando a frequência das palavras para incrementar as stopwords:
# DATA AND KI #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_daten_ki_de['corpus'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

# criando um dicionário compreensao:
most_common = {k: v for k, v in freq.items() if v > 100}

most_common

# len(german_stop_words) 232

{'daten': 586,
 'deutschland': 152,
 'dabei': 123,
 'wirtschaft': 132,
 'sowie': 253,
 'nutzung': 163,
 'bundesregierung': 312,
 'insbesondere': 143,
 'bereich': 139,
 'unternehmen': 147,
 'rahmen': 145,
 'maßnahmen': 125,
 'forschung': 136,
 'data': 129,
 'entwicklung': 173,
 'ki': 497}

In [7]:
#konjunktionen und pronomen, adverben
prep = ['dabei', 'sowie']

# redundante
de_woerte = ['deutschland', ]

german_stop_words.extend(prep)
german_stop_words.extend(de_woerte)

# len(german_stop_words) # 235

In [10]:
# Testando com o segundo banco:

# Clima #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_kl_timber_de['corpus'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

anpassung 137
maßnahmen 115
klimawandel 114
klimawandels 109
auswirkungen 105
entwicklung 88
folgen 87
kreislaufwirtschaft 87
bereits 67
holzwirtschaft 67
länder 61
deutsche 59
umwelt 54
holz 54
insbesondere 53
könnten 53
rahmen 52
sollten 51
bund 50
deutschen 47
bundesregierung 47
nutzung 45
etwa 43
anpassungsstrategie 41
ziele 41
deren 41
forschung 41
klima 39
veränderungen 39
arten 39
produkte 39
anpassungsmaßnahmen 38
ab 38
beitrag 37
mehr 37
materialien 37
besonders 36
daher 35
be 34
ressourcen 34
schutz 34
produkten 34
risiken 33
neue 33
klimaschutz 33
hinaus 32
möglichen 31
regionen 31
klimafolgen 31
müssen 31
klimaänderungen 30
regionalen 30
zusammenarbeit 30
kap 29
möglich 29
beispiel 29
lassen 29
wirtschaft 29
verantwortung 28
sollen 28
ländern 27
darüber 27
umsetzung 27
gen 27
beispielsweise 27
vgl 27
ls 27
bzw 26
sowohl 26
bedeutung 26
jahr 26
wichtige 26
globalen 25
unternehmen 25
ge 24
politik 24
strategie 23
weltweit 23
nutzen 23
bundesländern 23
bereich 23
derzeit 23
zi

In [11]:
# Testando com o terceiro banco:

# regierung #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_reg_de['corpus'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

stärken 212
setzen 155
unterstützen 154
schaffen 111
insbesondere 102
dafür 100
rahmen 90
europäischen 85
fördern 80
ländern 78
mehr 76
menschen 76
eu 75
verbessern 72
ziel 70
sollen 69
ermöglichen 67
maßnahmen 65
müssen 59
kommunen 58
deutschen 56
digitalisierung 53
gemeinsam 52
unserer 52
digitale 52
prüfen 52
bauen 52
arbeit 50
stellen 49
förderung 49
ab 48
internationalen 47
europäische 47
sicherheit 44
zusammenarbeit 44
beim 44
bund 44
entwickeln 44
bundesregierung 43
weiterentwickeln 43
sorgen 42
neue 42
ausbau 42
umsetzung 41
führen 41
neuen 41
unternehmen 41
wirtschaft 40
öffentlichen 40
erhöhen 40
nutzen 40
schutz 39
bundes 39
verantwortung 38
zugang 38
stärker 38
forschung 37
streben 37
grundlage 37
bildung 35
digitalen 35
nationalen 35
finanzierung 35
beschleunigen 34
unterstützung 34
kinder 34
einsatz 34
einsetzen 33
künftig 33
frauen 32
europäischer 32
entwicklung 32
demokratie 31
mittel 31
nationale 31
stärkung 31
ebene 31
besser 30
deutsche 30
verfahren 30
rahmenbedingun

In [12]:
german_stop_words.extend(['dafür'])

In [13]:
with open("german_bund_stopwords.json", "w") as f:
    json.dump(german_stop_words, f)

# # Carregue a lista a partir do arquivo
# with open("lista.json", "r") as f:
#     lista_carregada = json.load(f)  